<a href="https://colab.research.google.com/github/Natalie-Jones/IceCube-Data-Analysis/blob/master/Workbook_diffuse_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solutions #

We'll paste solutions here every 30 minutes

If you get stuck, raise your hand and we'll help you out individually


In [0]:
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import json
from google.colab import drive
drive.mount('/content/drive')

In [0]:
json_contents = json.load(open("/content/drive/My Drive/hese_toy_simulation.json", "r"))
simulation_mapping = json_contents["mapping"]
simulation_events = np.array(json_contents["events"])
del json_contents

In [0]:
sim_map = sorted(simulation_mapping.items(), key=lambda x: x[1])
print (sim_map)

In [0]:
print (simulation_events[:2])
[[(k, e[i]) for k, i in sim_map] for e in simulation_events[:2]]

In [0]:
json_contents = json.load(open("/content/drive/My Drive/hese_toy_data.json", "r"))
data_mapping = json_contents["mapping"]
data_events = np.array(json_contents["events"])
del json_contents

In [0]:
data_map = sorted(data_mapping.items(), key=lambda x: x[1])
print (data_map)

In [0]:
print (data_events[:2])
[[(k, e[i]) for k, i in data_map] for e in data_events[:2]]

In [0]:
energy_bins = np.logspace(np.log10(60.0e3), np.log10(1.0e7), 20+1)
zenith_bins = np.arccos(np.linspace(-1, 1, 10+1))[::-1]
topology_bins = np.linspace(0, 2, 2+1)

h = np.histogram(data_events[:,0], bins = energy_bins)

In [0]:
centers = (energy_bins[1:] + energy_bins[:-1])/2.
lower = centers - energy_bins[:-1] 
upper = energy_bins[1:] - centers
xerr = np.array([lower, upper])
yerr = np.sqrt(h[0])
yerr = np.where(yerr==1, 0.99, yerr)

uplims = np.zeros(len(centers))
plt.figure(figsize=[10,6])
plt.errorbar(centers, h[0], xerr=xerr, yerr=yerr,capsize=4,fmt='o',color='k',alpha=0.9,linewidth=2,label='data')

plt.semilogy()
plt.semilogx()
plt.ylim(1e-1, 1e2)
plt.legend(fontsize=17)
plt.xlabel('Energy (GeV)',fontsize=15)

In [0]:
def weight_event(event, atmo_norm, astro_norm, astro_gamma):
    astroWeight = event[...,0]
    atmoWeight = event[...,1]
    primaryEnergy = event[...,3]
    
    weight = (atmoWeight * atmo_norm 
              + astroWeight * astro_norm * pow(primaryEnergy/1e5, -(astro_gamma-2.5)))
    
    return weight

In [0]:
energy_bins = np.logspace(np.log10(60.0e3), np.log10(1.0e7), 20+1)
zenith_bins = np.arccos(np.linspace(-1, 1, 10+1))[::-1]
topology_bins = np.linspace(0, 2, 2+1)

In [0]:
def make_bin_masks(energies, zeniths, topologies,
                   energy_bins=energy_bins, zenith_bins=zenith_bins, topology_bins=topology_bins):
    
    assert(len(energies) == len(zeniths))
    assert(len(energies) == len(topologies))
    
    n_energy_bins = len(energy_bins) - 1
    n_zenith_bins = len(zenith_bins) - 1
    n_topology_bins = len(topology_bins) - 1
    
    energy_mapping = np.digitize(energies, bins=energy_bins) - 1
    zenith_mapping = np.digitize(zeniths, bins=zenith_bins) - 1
    topology_mapping = np.digitize(topologies, bins=topology_bins) - 1
    bin_masks = []
    for i in range(n_topology_bins):
        for j in range(n_zenith_bins):
            for k in range(n_energy_bins):
                mask = topology_mapping == i
                mask = np.logical_and(mask, zenith_mapping == j)
                mask = np.logical_and(mask, energy_mapping == k)
                bin_masks.append(mask)
    return bin_masks

In [0]:
bin_masks = make_bin_masks(simulation_events[:,2], simulation_events[:,6], simulation_events[:,5])
data_masks = make_bin_masks(data_events[:,0], data_events[:,2], data_events[:,1])

In [0]:
def get_expectation(events, masks, weighting):
    all_weights = weighting(events)
    weights = []
    for mask in masks:
        weight = np.sum(all_weights[mask])
        weights.append(weight)
    return np.array(weights)


In [0]:
def llh(data, simulation_events, atmo_norm, astro_norm, astro_gamma):
    expect = get_expectation(simulation_events,
                             bin_masks,
                             lambda e: weight_event(e, atmo_norm, astro_norm, astro_gamma))
    l = -(data * np.log(expect) - expect - sp.special.loggamma(data+1))
    l[np.logical_and(data == 0, expect == 0)] = 0
    l = np.sum(l).real
    return l

In [0]:
data = np.sum(np.array(data_masks), axis=1)
print (llh(data, simulation_events, 1.0, 1.0, 2.5))

222.27828015190465


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


In [0]:
res = sp.optimize.minimize(lambda x: llh(data, simulation_events, *x),
                           [1.0, 6.0, 2.9],
                           method='L-BFGS-B',
                           bounds=[[0,None],[0,None],[None,None]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


In [0]:
print (res)

      fun: 175.4846456749209
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([8.52651283e-06, 0.00000000e+00, 1.13686838e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 52
      nit: 11
   status: 0
  success: True
        x: array([0.9963915 , 6.71417868, 2.99387581])


In [0]:
print ('Atmospheric Normalization = ', res.x[0])
print ('Astrophysical Normalization = ', res.x[1])
print ('Astrophysical Gamma = ', res.x[2])

Atmospheric Normalization =  0.996391503118721
Astrophysical Normalization =  6.714178676939703
Astrophysical Gamma =  2.993875806500643
